In [ ]:
# import necessary libraries
import os
import qiskit
import time
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
from torch import Tensor
from torch.nn import Linear, CrossEntropyLoss, MSELoss
from torch.autograd import Function 
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.transforms import Grayscale
from torch.utils.data import SubsetRandomSampler, DataLoader
from qiskit import QuantumCircuit
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap, ZFeatureMap
from qiskit_machine_learning.neural_networks import SamplerQNN, EstimatorQNN
from qiskit_machine_learning.connectors import TorchConnector
from qiskit.primitives import Estimator

In [ ]:
# import data

IMG_SIZE = (160, 160)
NUM_CLASSES = 4
BATCH_SIZE = 32

class CustomDataset(torch.utils.data.Dataset):
    """
    Custom dataset class for transforming labels into one-hot encoded vectors.
    """
    def __init__(self, dataset):
        """
        Initializes a new instance of the CustomDataset class.

        Parameters:
            - dataset (torchvision.datasets.ImageFolder): The original dataset.
        """
        self.dataset = dataset

    def __getitem__(self, index):
        """
        Retrieves the item at the specified index.

        Parameters:
            - index (int): Index of the item to retrieve.

        Returns:
            - image (torch.Tensor): The image tensor.
            - one_hot_label (torch.Tensor): One-hot encoded label tensor.
        """
        image, label = self.dataset[index]
        one_hot_label = torch.eye(NUM_CLASSES)[label]
        return image, one_hot_label

    def __len__(self):
        """
        Returns the length of the dataset.

        Returns:
            - len (int): Length of the dataset.
        """
        return len(self.dataset)

# Training dataset   
train_ds = datasets.ImageFolder(root=r"C:\Users\giuseppe.dambruoso\OneDrive - LUTECH SPA\Desktop\Progetto\Dataset\train",
                                transform=transforms.Compose([
                                    transforms.Resize(IMG_SIZE),
                                    transforms.Grayscale(num_output_channels=1),
                                    transforms.ToTensor()
                                ]))
train_ds = CustomDataset(train_ds) # Transform the original training dataset into a custom dataset with one-hot encoded labels
# train_loader = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
# train_subset_size = 96
# indices = torch.randperm(len(train_ds))[:train_subset_size]
# subset_sampler = SubsetRandomSampler(indices)
# train_loader = torch.utils.data.DataLoader(
#     train_ds,
#     batch_size=BATCH_SIZE,
#     sampler=subset_sampler
# )

# Validation dataset
validation_ds = datasets.ImageFolder(root=r"C:\Users\giuseppe.dambruoso\OneDrive - LUTECH SPA\Desktop\Progetto\Dataset\test",
                                transform=transforms.Compose([
                                    transforms.Resize(IMG_SIZE),
                                    transforms.Grayscale(num_output_channels=1),
                                    transforms.ToTensor()
                                ]))
validation_ds = CustomDataset(validation_ds) # Transform the original testing dataset into a custom dataset with one-hot encoded labels
# validation_loader = torch.utils.data.DataLoader(validation_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
# validation_subset_size = 32
# indices = torch.randperm(len(validation_ds))[:validation_subset_size]
# subset_sampler = SubsetRandomSampler(indices)
# validation_loader = torch.utils.data.DataLoader(
#     validation_ds,
#     batch_size=BATCH_SIZE,
#     sampler=subset_sampler
# )

# Combine the training and validation datasets
# combined_ds = ConcatDataset([train_ds, validation_ds])

# Define the sizes for the training and validation subsets
train_size = len(train_ds)
validation_size = len(validation_ds)

# Split the combined dataset into training and validation subsets
# train_ds, validation_ds = random_split(combined_ds, [train_size, validation_size])

# Create data loaders for the training and validation subsets
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
validation_loader = torch.utils.data.DataLoader(validation_ds, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

In [ ]:
# define the classical layers
def ConvLayer(in_channels, out_channels, kernel_size, padding='same'):
    """
    Helper function to create a convolutional block.

    Parameters:
        - in_channels: Number of input channels.
        - out_channels: Number of output channels.
        - kernel_size: Size of the convolutional kernel.
        - padding: Padding type for the convolution.

    Returns:
        - model_cb: Convolutional block as a Sequential module.
    """
    model_cb = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=padding),
        nn.BatchNorm2d(out_channels),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    return model_cb

def DenseLayer(in_units, out_units):
    """
    Helper function to create a dense block.

    Parameters:
        - in_units: Number of input units.
        - out_units: Number of output units.

    Returns:
        - model_db: Dense block as a Sequential module.
    """
    model_db = nn.Sequential(
        nn.Linear(in_units, out_units),
        nn.ReLU(),
        nn.BatchNorm1d(out_units),
        nn.Dropout(p=0.2)
    )
    return model_db

In [ ]:
# define the quantum layer
def create_quantum_layer(num_inputs, num_outputs, feature_map, quantum_layer, shots):
    feature_map = feature_map(num_inputs) # create a ZZFeatureMap
    ansatz = RealAmplitudes(num_inputs, reps=1) # create a RealAmplitudes map with one repetition

    quantum_circuit = QuantumCircuit(num_inputs) # create a quantum circuit
    quantum_circuit.compose(feature_map, inplace=True) # apply the feature map
    quantum_circuit.compose(ansatz, inplace=True) # apply the ansatz

    if quantum_layer == EstimatorQNN:
        quantum_layer = EstimatorQNN(
                            circuit=quantum_circuit,
                            input_params=feature_map.parameters,
                            weight_params=ansatz.parameters,
                            input_gradients=True,
                        )
        quantum_layer.estimator.set_options(shots=shots)
    
    if quantum_layer == SamplerQNN:
        parity = lambda x: "{:b}".format(x).count("1") % 2  # optional interpret function
        quantum_layer = SamplerQNN(
                            circuit=quantum_circuit,
                            input_params=feature_map.parameters,
                            weight_params=ansatz.parameters,
                            output_shape=num_outputs,
                            interpret=parity,
                        )
        quantum_layer.sampler.set_options(shots=shots)
    
    return quantum_layer

In [ ]:
# define the net
class Net(nn.Module):
    """
        Neural network model composed of convolutional, dense and quantum layers.

        Architecture:
        - Three convolutional layers: ConvLayer(1, 256), ConvLayer(256, 128), ConvLayer(128, 64)
        - Two fully connected layers: DenseLayer(25600, 128), DenseLayer(128, 64)
        - Output layer: nn.Linear(64, 4)
        - Quantum layer: QuantumLayer(backend, 100, np.pi/2)

        Parameters:
            - None

        Attributes:
            - conv1 (ConvLayer): First convolutional layer with input channels=1 and output channels=256.
            - conv2 (ConvLayer): Second convolutional layer with input channels=256 and output channels=128.
            - conv3 (ConvLayer): Third convolutional layer with input channels=128 and output channels=64.
            - fc1 (DenseLayer): First fully connected layer with input features=25600 and output features=128.
            - fc2 (DenseLayer): Second fully connected layer with input features=128 and output features=64.
            - fc3 (nn.Linear): Output layer with input features=64 and output features=4.
            - quant (QuantumLayer): Quantum layer with specified backend, shots, and shift parameters.

        Methods:
            - forward(x): Forward pass of the neural network.

        Usage:
        ```
        net = Net()
        output = net(input_tensor)
        ```
    """
    def __init__(self, quantum_layer):
        super(Net, self).__init__()
        self.conv1 = ConvLayer(1, 256, kernel_size=5)
        self.conv2 = ConvLayer(256, 128, kernel_size=5)
        self.conv3 = ConvLayer(128, 64, kernel_size=5)
        self.fc1 = DenseLayer(25600, 128)
        self.fc2 = DenseLayer(128, 64)
        self.fc3 = DenseLayer(64, 4)
        self.quant = TorchConnector(quantum_layer) # ha dimensioni 4x1 se Estimator, 4x4 se Sampler
        self.fc4 = nn.Linear(4, 4)
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.quant(x)
        x = self.fc4(x)
        return x

In [ ]:
# train the net
epochs = 30
shots = 5000
train_loss_list = []
train_accuracy_list = []
validation_loss_list = []
validation_accuracy_list = []
quantum_layer = create_quantum_layer(
    num_inputs=4,
    num_outputs=4,
    feature_map=ZZFeatureMap,
    quantum_layer=SamplerQNN,
    shots = shots
)
model = Net(quantum_layer)
LEARNING_RATE = 0.001
optimizer = optim.Adam(model.parameters(), LEARNING_RATE)
loss_func = nn.CrossEntropyLoss()

for i in range(epochs):
    start_time = time.time()
    model.train()
    total_loss = []
    correct = 0
    total_samples = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad(set_to_none=True)
        output = F.softmax(model(data).squeeze(), dim=-1)
        target = target.squeeze().type('torch.DoubleTensor')
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        total_loss.append(loss.item())
        pred = []
        for o in output:
            prediction = torch.tensor(o.argmax())
            pred.append(prediction)
        tar = []
        for t in target.squeeze():
            targ = t.argmax()
            tar.append(targ)
        for j in range(len(pred)):
            if pred[j] == tar[j] :
                correct += 1
    train_epoch_accuracy = correct / (len(train_loader)*BATCH_SIZE) * 100
    train_epoch_loss = sum(total_loss) / len(total_loss)
    train_accuracy_list.append(train_epoch_accuracy)
    train_loss_list.append(train_epoch_loss)
    end_time = time.time()
    epoch_time = (end_time - start_time)/60
    print('Epoch: {:d}, Loss: {:.4f}, Accuracy: {:.2f}%, Time: {:.1f}min'.format(i+1, train_epoch_loss, train_epoch_accuracy, epoch_time))

In [ ]:
# Plotting the training loss over epochs
plt.plot(train_loss_list, label='Training Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()
plt.show()

In [ ]:
model.eval()
with torch.no_grad():
    correct = 0
    for batch_idx, (data, target) in enumerate(validation_loader):
        output = model(data).squeeze()
        target = target.squeeze().type('torch.DoubleTensor')
        validation_epoch_loss = loss_func(output, target)
        pred = []
        for o in output:
            prediction = torch.tensor(o.argmax())
            pred.append(prediction)
        tar = []
        for t in target:
            targ = t.argmax()
            tar.append(targ)
        for l in range(len(pred)):
            if pred[l] == tar[l] :
                correct += 1
    validation_epoch_accuracy = correct/(len(validation_loader)*BATCH_SIZE)*100
    print('Validation - Loss: {:.4f}, Accuracy: {:.1f}%'.format(validation_epoch_loss, validation_epoch_accuracy))

In [ ]:
path = r'C:\Users\giuseppe.dambruoso\OneDrive - LUTECH SPA\Desktop\Progetto\Risultati\trained_HQCCNN_fc3_shuffled'
torch.save(model.state_dict(), path)